In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.decomposition import TruncatedSVD
from cut_sentence import segment

In [2]:
content = pd.read_csv('./movie_comments.csv', encoding='utf-8')
content = content.dropna(axis=0,how='any')
print(len(content),content)

130454             id                                        link  \
0            1  https://movie.douban.com/subject/26363254/   
1            2  https://movie.douban.com/subject/26363254/   
2            3  https://movie.douban.com/subject/26363254/   
3            4  https://movie.douban.com/subject/26363254/   
4            5  https://movie.douban.com/subject/26363254/   
5            6  https://movie.douban.com/subject/26363254/   
6            7  https://movie.douban.com/subject/26363254/   
7            8  https://movie.douban.com/subject/26363254/   
8            9  https://movie.douban.com/subject/26363254/   
9           10  https://movie.douban.com/subject/26363254/   
10          11  https://movie.douban.com/subject/26363254/   
11          12  https://movie.douban.com/subject/26363254/   
12          13  https://movie.douban.com/subject/26363254/   
13          14  https://movie.douban.com/subject/26363254/   
14          15  https://movie.douban.com/subject/26363254/   
1

In [3]:
model = Word2Vec.load("wiki.model")
word_vectors = model.wv
del model

In [4]:
import random
#1. 预处理---分词
data=[]
for item in content.iterrows():
#     print(item[1]['comment'])
    words = segment(item[1]['comment'],type='arr')
#     print(words)
    data.append({'comment':item[1]['comment'],'star':item[1]['star'],'words':words})

#2. 预处理---分解为训练集/验证集/测试集
total_size = len(data)
print(len(data))
train_size = int(0.7*total_size)
print(train_size)
valid_size = int(0.2*total_size)
test_size = int(0.1*total_size)
print("use train_size: %d, valid_size: %d, test_size: %d" % (train_size,valid_size,test_size))

random.shuffle(data)
train_set = data[:train_size]
valid_set = data[train_size:train_size+valid_size]
test_set = data[(train_size+valid_size):train_size+valid_size+test_size]

# def sentence_embedding(sentence,vocabulary):
#     pass

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\webberg\AppData\Local\Temp\jieba.cache
Loading model cost 0.952 seconds.
Prefix dict has been built succesfully.


130454
91317
use train_size: 91317, valid_size: 26090, test_size: 13045


In [5]:
from collections import defaultdict
# 尝试采用SIF sentence embedding(https://www.colern.com/paper/get?paperid=1bcb3f85-45ba-4f21-9609-4a598eb32cec)

def get_sentence_embedding(dataset):
    # 1.计算词频
    # unigram probability
    p_w=defaultdict(int)
    train_words_count  = 0

    for item in dataset:
        for word in item['words']:
            p_w[word]+=1
            train_words_count += 1
    
    # 初始句向量
    sentence_vectors=[]
    alpha = 0.001
    for sentence in dataset:
        sentence_len = len(sentence['words'])
    #     print(sentence_len)
    #         continue
    #     print(sentence['words'])
        sentence_vec=np.zeros(128)
        for word in sentence['words']:
            if word in word_vectors.vocab:
                sentence_vec += word_vectors[word]*(alpha/(alpha+p_w[word]/train_words_count))
            # 抛弃UNK词,看看效果

        if sentence_len==0:
            print(sentence['comment'])
            sentence_vectors.append(None)
        else:
            sentence_vectors.append(sentence_vec/sentence_len)
            
    # 剔除表情文字及其他长度为0的句子
    new_data_set = pd.DataFrame({'sentence':sentence_vectors,'star':[item['star'] for item in dataset]})
#     print(new_data_set.shape)
    new_data_set = new_data_set.dropna(axis=0,how='any')
#     print(new_data_set.shape)
    
    svd = TruncatedSVD(n_components=1,n_iter=5,random_state=0)
    #将所有求得的初始句向量做成矩阵
    print(new_data_set.shape[0],new_data_set.shape[1])
    sentence_x = np.zeros((new_data_set.shape[0],len(sentence_vectors[0])))
    for i in range(new_data_set.shape[0]):
        sentence_x[i,:]=new_data_set['sentence'].iloc[i]
#     print(sentence_x)
    svd.fit(sentence_x)
    pc = svd.components_
    #计算每个句子的句向量
    embed = sentence_x-sentence_x.dot(pc.transpose()) * pc
#     print(embed.shape)
    return embed,new_data_set

In [6]:
#计算每个句子的句向量
embed,new_train_set = get_sentence_embedding(train_set)
print(embed.shape)

2013.4.14
quite funny~~
2006.11.17
2.23
2007.11.15 CCTV6 home
TAKE A BOW
2014.12
·
2.5
HAHAHAHA~
Ellen Page, you are Hollywood's future!
FREEEEEEEEEEEEEEEDOM
124
2012.12.25
Pink...
Blablablablablablablablablabla
You are the best.
http://www.tudou.com/programs/view/D3-6VCqcXeE/
1997
。。。
God，where are you？ Show your mercy！
x
Love me, if you dare...
not my type
moon river, audrey hepburn
Fast.And.Furious.DVDRip.XviD-JUMANJi
My Review http://www.bilibili.com/video/av3997994/
Magicians do not exist. But the ivy leaf is still there.
7.5/7.6
GATEWAY  201111
All about life
x
Aaliyah is soo fine!!!
~~
warm
http://www.tudou.com/programs/view/05HbV-9dBSQ
classic dialog.
she‘s so charming~
jason statham's movie
7.8
so fuck them,ok?
my fav
lan
http://movie.douban.com/review/4476174/
7.1；on va où, maintenant?
4.5.
speak out！
117
"It's funny to be doing this on 60. I just hope my knees hold up!"
in sama
MINI COOPER SHOW
inspirational
"I have to find my husband I have to find my husband"
B-
。。。
2006.9

x
QMDb：6/10
My touch moment.
x
V!
ROSE
this is actually very slow @@"
It  is  life
:)
punk is not dead！
。。
Ride or die
nothing is mean to be
6.8/7.3
2010//OVA
some one s watching you
we still love young virgin not blowing anyone out
summer
DVD
=。=
= =
FIN
winner takes all
obsession
cult.
boring
so cool
2008.5.3
so so
all or nothing
7.0/7.7
he's so cute
http://www.douban.com/people/hitchitsch/status/1784731651/
7.0/7.0
http://www.angryalien.com/0406/reservoirbuns.asp
I'llonly say thisonce 
　　I've never said it before 
　　But this kind of certainty comes justonce in a lifetime
charm little film, watching lovely people on screen is so fun. competent direction is also a plus, but it's time to Reitman to step up
For Fiona！！
let's rock n roll!
x
it's written
Instead of offering a convincing argument against the death penalty, this implausible, convoluted thriller pounds the viewer over the head with its message.
soga……
http://v.youku.com/v_show/id_XMjIzMTk0MTAw.html
Megryan ！
DVDRip：http://ww

EXM？！！！！！！！
09/6/12
no rule
BRAVO!BINGO!
7.0/7.7
imagination is everything...
kinda cliche, one part missing, I don't understand why Jack got involved at the first time, not explained later.
based on a true story
2008-9-6
cross heart
I just can't figure out why they put "How Do I" as the ending song. Maybe many people have the same question so that's why the song was nominated for both an Oscar as Best Original Song and a Razzie as Worst Original Song. Funny trivia,lol.
masterpiece
2007.12.15
3.5
1
2011.09.18
fuck them all!
i wait.
Why do we fall,sir?So that we can learn to pick ourselves up.
Just because there are things I don’t remember doesn’t make my actions meaningless.
///
HIGH!!!~!~@~!@~!@~!
@HBO
4.5
TT
best chick flick of the year
2009-9-6 19:47:22 7/6.9(522)
oceans back
brilliant!!!!
Everybody has a box. I've got a lot.
u got me
Based on a true story
http://so.youku.com/search_video/q_%E8%A5%BF%E8%A5%BF%E9%87%8C%E7%9A%84%E7%BE%8E%E4%B8%BD%E4%BC%A0%E8%AF%B4%20%E6%9C%AA%E5%88%A0

Story is fine, not deep though, but still interesting enough,it's also clear and fast. animation making is excellent, music is good..  above all this anime worth watching
COOL
2.11human beings are a disease.
I'm trying to think of a movie that has more F word than this one, despite anything Robert De Niro was in, I don't know, Alpha Dog?
shit～
http://www.tudou.com/albumplay/z7-L0DJtClA.html
2006.10.15
funny and entertaining heist movie!
I will follow him。。。
2008.1.11
easy to forget movie (watched on the plane)
I love kevin spacey
sunshine!!!!!!
2008-1-31 7/6.5(329)
24
waiting for you till the end of my life
Life is not the amount of breaths you take, it's the moments that take your breath away.
ARE U ENJOY THE TIME OF EVE?
x
2007.4.4/5
sosososososososo
781M  .BDRip.MiniSD-TLF.mkv
3.7
fucking cool
So many insanity and too many "fxxking".
http://open.vistastory.com/mobilecms//sharelink.action?id=9143
SB
007
oh,my love.
boring
armchannel
Forgetful Lucy
never let it down
http://movie.douba

it's life, beautiful life
texex……
see you again
If you have that much money to spend, put it into SAVING THIS PLANET first.
4*-
20100905 jtl
7.2/10
ccav.
the suprasternal notch
아보지TTTTTTTTTT
……
2008 5 28
Q-Q
oh no！
extraordinarily excellent！！
sucks
91
2012.12.14 22:53
Be There or Be Square
x
r u kidding me
We'll always have Paris.
It’s not who you are underneath, but it's what you do that defines you
freedom
matt! matt! matt!
for Paul.
really for idiots..
95
What's this?
just soso
The best love story I've ever seen.
MoonRiver
2012
tomoyo~tomoyo~tomoyo~tomoyo~tomoyo~tomoyo~tomoyo~tomoyo~tomoyo~
2.9
KUSO
My guilty pleasure!
http://www.bighead.cn/?p=34
ヽ(´o｀；
Love is all around....
2006.11.15
cool~
pretty cute
Mark
Neo, I believe. I know it wasn't a dream.
“But it doesn’t take much to see that the problems of three little people don’t amount to a hill of beans in this crazy world.”
111
just soso
zhuang bi!!!!!!!!!!!!!!
here is looking at you ,kid~
CHANGSHA TV
I'd like to join u by ur side

Dustin Hoffman is awesome！
Nic Cage playing John Travolta, meets John Travolta playing Nic Cage, in the middle of a thunderous John Woo shootout. The actors are separated by mirrored columns, allowing John Woo to get away with a stylish, psychologically-loaded sleight-of-hand that they are looking at their
No Country for Old Men
james! im a fan of u, but for decribing beginning with a c for the film
：（
106
。。。。。。。
2007.12.15
soso
what the hell
that is one hour i am never getting back。。。
anne
free？awake in the fiction, fall down in the reality?Or vice versa
http://www.tudou.com/programs/view/fqXT5SMBWoU/
I went to the woods because I wanted to live deliberately, I wanted to live deep and suck out all the marrow of life, and not when I had come to die, discover that I had not lived.
http://v.youku.com/v_show/id_XMzcyMzUzNzI=.html
3
incredibly cliche-free, funny and fresh all the way through. The best part? The explanation is as interesting as the crazy set-up
w
R U talking to me?
it's no

“Yippie Kai-Yay Motherfucker！！”
"Fuck Physics" is back!!! #Goes back and rewatch Mad Max again  #How come Furiosa doesn't get to drive
very violent
a bit mo lei tau
somehow it's very early 90s...
fucking cinema，fucking audience！
DVD
7.8/10
haha....hahahaha..haha.haha.haha.ha....hahahahaha......................ha...........hahahaha
it's about  how to build a life
にょろーん
you are my best adventure
good
media
Limit Cycle
O__O "…
You can always be a worse version if 'him',or a better version if self.
！！
7.0/7.0
Never say die.
。。。
x
dirty, rich and famous stuffs
V5
where is my miss Ho?????????????????
http://v.youku.com/v_show/id_XMTEzNjQ4ODY4.html
Does he make you laugh? He doesn't make me cry.
Luckey
My name is Harvey Milk, and I'm here to recruit you!
8.6/10
An amazing story,a wonderful story.
Just love football for no reason. (Now I look back, yes, there is a reason. Ben. I didn't understand how much I liked him until later on. 8/10/2016.)
6.8
just so so
fuck！
there's no other way
2011111

。。。。。
Up.In.The.Air.2009.DVDRip.XviD-Larceny
This is fucking AMAZING!
3
OK
PS
Touching
iron
GJ！
New York, NY!
3---
080122
i see u
ColinFirth！！！
1993
This is awesome! I love you Gru~!
like a virgin
2014.05.10
blahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblahblah
。。
★★★☆
team cap
！
just for money~~~
http://movie.douban.com/review/7360621/
woooooooooooooooooooooooooow
j-anime lives best
7.5/10
And in the end、the greatest thief of all is time...
haomei
If you are big on visuals, styles and details. This  is definately the one to watch. But storyline is still a long way away from Sin City.
THOMAS~
3.8
Courage is not the absence of fear but rather the judgment that something else is more important than fear.
my first time met mammoth
so so
impressive match
crazy and simple story.
12Aug
James Mc Avoy
sucks
2003.3.15 cctv6
GIGI
"

In [7]:
print(train_set[5])
#查看句向量在词向量空间中的临近词
word_vectors.similar_by_vector(embed[5])

{'comment': '2013.4.14', 'star': '4', 'words': []}


[('该', 0.5967848300933838),
 ('因该', 0.4814724326133728),
 ('此', 0.46952563524246216),
 ('喷上', 0.4658101201057434),
 ('而此', 0.456278920173645),
 ('涂抹', 0.443098783493042),
 ('这一', 0.4203059673309326),
 ('移走', 0.41832053661346436),
 ('这块', 0.41286128759384155),
 ('Kilise', 0.4072677493095398)]

#### 开始构建神经网络

In [8]:
X = torch.tensor(embed,dtype=torch.float32,requires_grad=True)
print(X.dtype)
y = torch.tensor(new_train_set['star'].values.astype(int)-1,dtype=torch.long)
print(torch.max(y),torch.min(y))
print(y.dtype)
print(len(X),len(y))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
input_size = 128
hidden_size = 1024
num_classes = 5
learning_rate = 0.001

torch.float32
tensor(4) tensor(0)
torch.int64
88738 88738
cuda


In [9]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size, 2048)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc3 = nn.Linear(2048, 4096)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=0.6)
        self.fc4 = nn.Linear(4096, 2048)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(p=0.5)
        self.fc5 = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.dropout3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        out = self.dropout4(out)
        out = self.fc5(out)
        return out

In [10]:
epoch_size = 201
batch_size = 512
total_step = int(len(y)/batch_size)

model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=0.0002)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=50,gamma=0.1)

losses = []
for epoch in range(epoch_size):
    start = 0
    end = 0
    for i in range(total_step):
        start = end
        end = start + batch_size
        
        labels = y[start:end].to(device)
        inputs = X[start:end].to(device)
        
        outputs = model(inputs)
#         print(outputs.shape)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
#         scheduler.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epoch_size, i+1, total_step, loss.item()))
            losses.append(loss.item())
    scheduler.step()
     

Epoch [1/201], Step [100/173], Loss: 1.4157
Epoch [2/201], Step [100/173], Loss: 1.3787
Epoch [3/201], Step [100/173], Loss: 1.3883
Epoch [4/201], Step [100/173], Loss: 1.3708
Epoch [5/201], Step [100/173], Loss: 1.3708
Epoch [6/201], Step [100/173], Loss: 1.3438
Epoch [7/201], Step [100/173], Loss: 1.3489
Epoch [8/201], Step [100/173], Loss: 1.3451
Epoch [9/201], Step [100/173], Loss: 1.3317
Epoch [10/201], Step [100/173], Loss: 1.3236
Epoch [11/201], Step [100/173], Loss: 1.3343
Epoch [12/201], Step [100/173], Loss: 1.3030
Epoch [13/201], Step [100/173], Loss: 1.3273
Epoch [14/201], Step [100/173], Loss: 1.2939
Epoch [15/201], Step [100/173], Loss: 1.3105
Epoch [16/201], Step [100/173], Loss: 1.2978
Epoch [17/201], Step [100/173], Loss: 1.2652
Epoch [18/201], Step [100/173], Loss: 1.2705
Epoch [19/201], Step [100/173], Loss: 1.2878
Epoch [20/201], Step [100/173], Loss: 1.2511
Epoch [21/201], Step [100/173], Loss: 1.2602
Epoch [22/201], Step [100/173], Loss: 1.2619
Epoch [23/201], Ste

Epoch [182/201], Step [100/173], Loss: 0.8606
Epoch [183/201], Step [100/173], Loss: 0.8873
Epoch [184/201], Step [100/173], Loss: 0.8994
Epoch [185/201], Step [100/173], Loss: 0.9002
Epoch [186/201], Step [100/173], Loss: 0.8865
Epoch [187/201], Step [100/173], Loss: 0.8937
Epoch [188/201], Step [100/173], Loss: 0.9445
Epoch [189/201], Step [100/173], Loss: 0.8915
Epoch [190/201], Step [100/173], Loss: 0.8904
Epoch [191/201], Step [100/173], Loss: 0.9376
Epoch [192/201], Step [100/173], Loss: 0.9037
Epoch [193/201], Step [100/173], Loss: 0.9099
Epoch [194/201], Step [100/173], Loss: 0.8883
Epoch [195/201], Step [100/173], Loss: 0.9208
Epoch [196/201], Step [100/173], Loss: 0.8558
Epoch [197/201], Step [100/173], Loss: 0.9284
Epoch [198/201], Step [100/173], Loss: 0.8997
Epoch [199/201], Step [100/173], Loss: 0.9343
Epoch [200/201], Step [100/173], Loss: 0.8944
Epoch [201/201], Step [100/173], Loss: 0.9023


In [11]:
import matplotlib.pyplot as plt
plt.plot(losses)

In [12]:
def display_accuracy(inputs,labels,model):
    with torch.no_grad():
        correct = 0
        total = 0
        y = labels.to(device)
        output = model(inputs.to(device))
        _, predicted = torch.max(output, 1)
        total = y.size(0)
        correct = (predicted == y).sum().item()
        print('Accuracy: {} %'.format(100 * correct / total))

In [13]:
display_accuracy(X,y,model)

Accuracy: 63.48013252496112 %


In [14]:
# 验证集
valid_embed,new_valid_set = get_sentence_embedding(valid_set)        
print(len(valid_embed),len(new_valid_set))
valid_X = torch.tensor(valid_embed,dtype=torch.float32)
valid_labels = torch.tensor(new_valid_set['star'].values.astype(int)-1,dtype=torch.long)

cool
~
as what i've told everything begins with green hands and yup`this one boring
KEANU DRIVE BUS
so cool
I went to the woods because I wanted to live deliberately, I wanted to live deep and suck out all the marrow of life, and not when I had come to die, discover that I had not lived.
Best biography!
anti-climax
You are not an asshole, but you are trying to be.
Are you living in the real word?
There is no technique, no opponent.
Chose not to chose life.
Fav forever!!

Ewan is fucking adorable!!!!!!
decent
good
http://www.verycd.com/topics/9841/
8.2；will the lost souls be forgot?
Sweet~
@20120413
P.I.G
keep running keep killing
cool!
Queen's English
Are you lonesome tonight
right!there is nothing wrong with homosexual!
!!!
hope can set you free～
4.5.
Death.At.A.Funeral.LiMiTED.DVDRip.XviD-NeDiVx
What Thurman, Fox and Liu accomplish here is arguablly more difficult than playing the nuanced heroine of a Sundance thumb-sucker. There must be presence, physical grace, strength, personalit

personal taste issue
overrated
ccav-6
After summer, its autumn
old man my ass!
quite nice，want to see the previous one :)
79/52 BESTfilm
Finally,only Happy Together.
Beautiful! Beautiful!! Jacques Tati would be proud!
TAKE A BOW
http://movie.douban.com/review/2018350/
http://i.mtime.com/1076449/blog/4953496/
I like singing in rain!
I WANT TO RELAX
http://www.tudou.com/programs/view/D3-6VCqcXeE/
This is partially the reason I'm afraid of the uprising of AI so much.
oh cute……
A2C0D2G0M2=6
I def will get a sports car!
http://v.youku.com/v_show/id_XNDcwODcxMDA=.html
NOT BAD
sha  b
★★★★☆
???
the grand misery of the WWI Jews. A people bent down, a history darkened, a spirit lived.
Dubbed in English, super weird.
Not my type of anime, gave up watching it halfway.
You Are Not Alone
I don't want all by myself
7.2/7.1
Holy fucking craps!
。。。
If time could be traveled, I still wish you are there.
Jon Bon Jovi... = =|
lost and found of love
cool
u can have love even without a country.
Do you wanna

i like jane
ok i'll mimic`aa```a````a ``hahaha  ``
independent
ICS
truth and fiction
LOVE
。
1.4
Fear Can Hold You Prisoner, Hope Can Set You Free
2006.11.15
You Are Not Alone
3.5
if Liberatore met Moebius and could make an anime... it might look like this. pretty anarchic.
bored......
Benjamin..
61724
OVA
2008-9-6
.
is that elle also possible
waste my time
J Lo................................
3.5
7.8/10
Shia LaBeouf
Another manly movie! Russell Crowe is a real hero.
。。
You don't want to miss it!!!!
..
http://v.youku.com/v_show/id_XMzI3NTM2MA==.html
ははは
I just try to live everyday as if i can't deliberately come back for this one day. To enjoy it, as if it was the full final day of my extraordinary ordinary life.
Billy Elliot (2000) t7fcfb5ca9
"i know i'm awake but i feel like i'm in a dream."
x
1111
S&S Critics' Top Ten Poll
5.0。Witness me！
2008 9 13
claire danes bravo!!!
"It takes a wolf to kill a wolf"
★★★☆
great design reference
who knows what will the tide bring?so keep breathing.


In [15]:
# 验证集准确率
display_accuracy(valid_X,valid_labels,model)

Accuracy: 40.71912947484624 %


In [16]:
#测试集准确率
test_embed,new_test_set = get_sentence_embedding(test_set)
test_X = torch.tensor(test_embed,dtype=torch.float32)
test_labels = torch.tensor(new_test_set['star'].values.astype(int)-1,dtype=torch.long)

GATEWAY  250507
memory...
AMAZING
to live as a monster or to die as a good man?
Yea it reminds me of the move Peur Du Noir
8.0/7.2
Sometime I wish I had one of those.
Wooohooo～
2008.1.1
Rock & Roll save the Federation.
that's the power
2008 11 23
YY
The Longest Nite
best foodball movie ever
2007.7.7
3.5
God，I love this movie！
GOOD!
to me you are perfect
。。
"We appear and we disappear, and we're important to some, but we're just passing through. "
。。。
2006.10.15
......
Vincent Cassel is so sexy!~~~
"Dad died and shes happy because shes gonna dance."OMG
。。。
-You are not my type - Smart? - Single
what a cute couple!!
i n 55!W !
007
Royal Tramp?
Bradley Cooper❤
8
Release the baby!
the worst storyline, the worst music, the worst editing, the worst acting. a bunch of WTF.
CCTV6
beautiful.
just for SZU
Gainax creates the coolest animes in the universe.
not as bad as i thought.
CARPE DIEM
gay。。
oh~ cute spidy~
Magicians do not exist. But the ivy leaf is still there.
Here in my stomach. It's al

Russell Crowe o(≧v≦)o~~
2013.07.14
7.5/7.0
obviously move
OVA
kill is love
……
12683 2


In [17]:
display_accuracy(test_X,test_labels,model)

Accuracy: 40.605534968067495 %
